In [1]:
## Integrate Jacob's tidied up alignments

seq_blacklist = {}# {'alloascoidea_hylecoeti__OG1299__0_2015'}

In [2]:
import os
import pickle 
import pandas as pd

from Bio import SeqIO, AlignIO
#from Bio import codonalign

    

y1000plus_dir = os.path.normpath('C:/Users/heineib/Documents/GitHub/y1000plus_tools/data') + os.sep
#base_dir = os.path.normpath('G:/My Drive/Crick_LMS/projects/diverse_yeasts')


#Load y1000 species table
y1000_species = pd.read_csv(y1000plus_dir + os.path.normpath("y1000plus_tools_data/y1000plus/y1000_species_table.csv"), index_col=0)


orig_genome_lookup = dict(zip(y1000_species['original_genome_id'],y1000_species['old_species_names']))

base_dir = os.path.normpath('G:/My Drive/Crick_LMS/projects/diverse_yeasts/alphafold')


spec_abbrev_to_orig = {'Scer': 'Saccharomyces_cerevisiae', 
                    'Calb': 'Candida_albicans'
                   }

gene_id_2_y1000_id = {}

#Load S.cer lookup table: 
scer_lookup_fname = y1000plus_dir + os.path.normpath('y1000plus_tools_data/y1000plus/id_lookups/saccharomyces_cerevisiae.csv')
scer_lookup = pd.read_csv(scer_lookup_fname, index_col=0)
gene_id_2_y1000_id['Scer'] = dict(zip(scer_lookup.index,scer_lookup['y1000_id']))

#Load C.alb lookup table
calb_lookup_fname = y1000plus_dir + os.path.normpath('y1000plus_tools_data/y1000plus/id_lookups/candida_albicans.csv')
calb_lookup = pd.read_csv(calb_lookup_fname, index_col=0)
gene_id_2_y1000_id['Calb'] = dict(zip(calb_lookup.index,calb_lookup['y1000_id']))

#Lookup from swissprot_id to gene_id
swissprot_id_2_gene_id= {}
scer_swissprot_id_2_gene_id_df = pd.read_table(base_dir + os.sep + os.path.normpath('msas/structural/Scer_protein_names.tsv'))
swissprot_id_2_gene_id['Scer'] = dict(zip(scer_swissprot_id_2_gene_id_df['Swiss-Prot'],scer_swissprot_id_2_gene_id_df['OLN']))

calb_swissprot_id_2_gene_id = pickle.load(open(base_dir + os.sep + os.path.normpath('msas/structural/Mapping_calb.pkl'),"rb"))
swissprot_id_2_gene_id['Calb'] = dict(zip(calb_swissprot_id_2_gene_id.values(),calb_swissprot_id_2_gene_id.keys()))





In [3]:
def seq_squeeze(seq_in):
    seq_out = ''
    for res in seq_in:
        if res!= '-':
            seq_out = seq_out + res
    
    return seq_out

In [2]:
#Start with example: CDC19, 'OG1299'
#Open sequence alignment
#Open structural alignment

#Assign Codons to Sequences
#https://jlsteenwyk.com/PhyKIT/usage/index.html#protein-to-nucleotide-alignment

#Need to know which parts of each sequence was trimmed.  (could use clipkit format)
#col1: position in the alignment (starting at 1)
#col2: reports if site was trimmed or kept (trim or keep, respectively)
#col3: reports if the site is parsimony informative or not (PI or nPI, respectively)
#col4: reports the gappyness of the position (number of gaps / entries in alignment)

#I think I might need to do this in a custom way




##Data file: REF_Scer_AF-P00549-F1-model_v2.pkl



In [4]:
#Make Codon Based structural Alignments: 


#For an orthogroup/reference
struct_aln_aa_dir = base_dir + os.sep + os.path.normpath("msas/structural/fasta_renamed_struct_filt") + os.sep

#Loop through all structures in that directory
struct_aln_aa_fname_base =  'OG1299__Scer_AF-P00549-F1-model_v2.struct_filt.fasta'
struct_aln_aa_fname =  struct_aln_aa_dir + struct_aln_aa_fname_base #Use this as a filter for the structures to keep, also use to check aa alignment

struct_aln_aa = AlignIO.read(struct_aln_aa_fname, 'fasta')

struct_aln_aa_dict = {}
for record in struct_aln_aa:
    struct_aln_aa_dict[record.id] = str(record.seq)


(og,ref_ext) = struct_aln_aa_fname_base.split('__')
(ref, ext1,ext2) = ref_ext.split('.')

struct_aln_codon_fname =  base_dir + os.sep + os.path.normpath("msas/structural/fasta_renamed_struct_filt_codon/" + og + '__' + ref + '_struct_filt_codon.fasta')

struct_data_fname = base_dir + os.sep + os.path.normpath('/Output/Summary/REF_' + ref + '.pkl')

og_struct_data = pickle.load(open(struct_data_fname, 'rb'))

seq_aln_aa_fname = base_dir + os.sep + os.path.normpath("msas\ogs_pep_aligned\\" + og + '.mfaa.mafft' )
seq_aln_codon_fname = base_dir + os.sep + os.path.normpath("msas\ogs_cds_threaded\\" + og + '.mfna.threaded' )

seq_aln_aa = AlignIO.read(seq_aln_aa_fname, 'fasta')
seq_aln_aa_inds = {}

for jj, record in enumerate(seq_aln_aa):
     seq_aln_aa_inds[record.id] = jj
    
seq_aln_codon = AlignIO.read(seq_aln_codon_fname, 'fasta')
seq_aln_codon_inds = {}

for jj, record in enumerate(seq_aln_codon):
     seq_aln_codon_inds[record.id] = jj
    


#For each sequence
with open(struct_aln_codon_fname, 'w') as fout_struct_codon: 
    for seq_ind, name in enumerate(og_struct_data['Molecules'].keys()): 
        # seq_ind = 7
        # name = 'ascoidea_rubescens__OG1299__6_3105'
        spec_abbrev_test = name.split('_')[0]
        seq_keep = True
        if name in seq_blacklist:  #Make sure I am using Jacob's most recent sequence alignments
            seq_keep=False
            
        if spec_abbrev_test in {'REF', 'Spom', 'Calb'}: #it is never Scer as REF refers to the fact that we are using Scer as the reference species. 
            if spec_abbrev_test == 'REF': 
                spec_abbrev = 'Scer'
                name = '_'.join(name.split('_')[1:]) # removes 'REF_' from the beginning of the name
            else: 
                spec_abbrev = spec_abbrev_test 

            if spec_abbrev == 'Spom': 
                seq_keep = False
            else: 
                spec_old = spec_abbrev_to_orig[spec_abbrev]

                swissprot_id = name.split('_')[1].split('-')[1]
                y1000_id = gene_id_2_y1000_id[spec_abbrev][swissprot_id_2_gene_id[spec_abbrev][swissprot_id]]

        else: 
            (spec, og_name, y1000_id) = name.split('__')
            spec_old = orig_genome_lookup[spec] 

        if seq_keep: 
            seq_aln_id = spec_old + '|' + y1000_id
            if seq_aln_id in struct_aln_aa_dict.keys():  #Only included if it is part of Oliver's structural alignment
                #Extract protein sequence
                seq_aa_gaps = seq_aln_aa[seq_aln_aa_inds[seq_aln_id]].seq
                seq_aa = seq_squeeze(seq_aa_gaps)

                #Extract codon sequenc
                seq_codon_gaps = seq_aln_codon[seq_aln_codon_inds[seq_aln_id]].seq
                seq_codon = seq_squeeze(seq_codon_gaps)

                #Extract trimming data
                seq_mapping = og_struct_data['Mapping']['Mapping'][seq_ind]

                #Make aa_line and codon_line

                line_header = '>' + seq_aln_id + '\n'

                aa_line = ''
                codon_line = ''
                for aln_ind in seq_mapping: 
                    if aln_ind ==-1: 
                        aa_line = aa_line + '-'
                        codon_line = codon_line + '---'
                    else: 
                        aa_line = aa_line + seq_aa[aln_ind]
                        codon_line = codon_line + seq_codon[3*aln_ind: 3*aln_ind + 3]
                
                assert aa_line==struct_aln_aa_dict[seq_aln_id], seq_aln_id + ' msa in filtered structure alignments does not have same alignment as corresponding sequence, ' + name + ', in raw data'
                   
                fout_struct_codon.write(line_header)
                fout_struct_codon.write(codon_line + '\n')
                


KeyError: 'Saccharomyces_cerevisiae|110_4286'

In [8]:
swissprot_id

'P00549'

In [ ]:
emboss: 
http://emboss.open-bio.org/rel/rel6/apps/coderet.html

In [10]:
seq_aln_aa_inds

{'Alloascoidea_hylecoeti|0_2015': 0,
 'Alloascoidea_hylecoeti|0_6964': 1,
 'Alloascoidea_hylecoeti|0_7561': 2,
 'Ogataea_parapolymorpha|104_2044': 3,
 'Pachysolen_tannophilus|106_930': 4,
 'Tortispora_caseinolytica|134_405': 5,
 'Torulaspora_delbrueckii|135_4439': 6,
 'Vanderwaltozyma_polyspora|136_2033': 7,
 'Wickerhamomyces_anomalus|139_1240': 8,
 'Candida_apicola|13_3972': 9,
 'Yarrowia_lipolytica|144_6037': 10,
 'yHMPu5000034604_Sporopachydermia_lactativora|170_306': 11,
 'yHMPu5000034604_Sporopachydermia_lactativora|170_6188': 12,
 'yHMPu5000034957_Hanseniaspora_osmophila|247_3065': 13,
 'Candida_tropicalis|30_3271': 14,
 'Zygosaccharomyces_rouxii|342_99': 15,
 'Cyberlindnera_jadinii|35_1953': 16,
 'Debaryomyces_hansenii|36_3767': 17,
 'Eremothecium_gossypii|40_309': 18,
 'Geotrichum_candidum|43_1496': 19,
 'Geotrichum_candidum|43_2944': 20,
 'Kazachstania_naganishii|49_3029': 21,
 'Kazachstania_naganishii|49_4108': 22,
 'Kluyveromyces_lactis|50_393': 23,
 'Kluyveromyces_marxianus

In [9]:
name

'Scer_AF-P00549-F1-model_v2'

In [5]:
seq_aln_id

'Saccharomyces_cerevisiae|110_4286'

In [ ]:
# Prepare to Run CodeML
# Convert codon alignment to phylip, shorten names

# Get a tree (prune to only have correct leaves, use short names)

# Build codeml run file, set up input/output directories

In [ ]:
codeml_input = [('seqfile', 'OG1299_cds_aln.phy'), 
                ('treefile', 'OG1299_clean.treefile'), 
                ('outfile', 'pamlOne.out'), 
                ('noisy', '9'), #0,1,2,3,9: how much rubbish on the screen
                ('verbose', '1'), # 0: concise; 1: detailed, 2: too much
                ('runmode', '0'), #user tree
                ('seqtype', '1'), #Codons
                ('CodonFreq', '2'), #0:1/61 each, 1:F1X4, 2:F3X4, 3:codon table 
                ('clock', '0'), # No clock - rates vary from branch to branch
                ('model', '0'), #How many omegas estimated 0: 1, 1: one per branch,  2: 2 or more - need to specify branches 
                ('NSsites', '0'), #one
                ('icode', '0'), # universal code.  Yeast mitochondria is 3, Alternative yeast nuclear should be 12, but the doc says 8.
                ('fix_kappa', '0'), #Uses Kappa estimate as initial guess
                ('kappa', '2'), 
                ('fix_omega', '0'), 
                ('omega', '1'), # initial or fixed omega - for codons or codon based AAs - default is 0.4
                ('fix_alpha', '1'), # setting this along with NSsites and alpha=0 below means that NSsites is used
                ('alpha', '.0'),   #why .0 instead of 0?
                ('Malpha', '0'),  #one gamma distribution across all sites
                ('ncatG', '10'),  #Number of categories in dG of NSsites models - default is 3
                ('getSE', '0'),   #get standard errors of estimates
                ('RateAncestor', '1'),  #Ancestral state - default is 0
                ('method', '0'), #simultaneous
                ('cleandata', '1'), #remove sites with ambiguity data - yes (default is no)
                ('Small_Diff', '.5e-6')  #Use a value between 1e-8 and 1e-5 and check that the results are not sensitive to the value used.
               ]

for (param_name, param_val) in 

In [ ]:
#function: run codeML given a codon alignment, output into file

In [ ]:
#function: #Extract relevant information from CodeML file
#Overall Ev Rate

In [ ]:
#Compare Ev Rate for seq alignment, structural alignment, all structurally conserved regions, sub-network

'ATGGTGGCCAACGGATACCACCATAAACTGAGCCAGCGTCTGAGCCAGCGTCTGAGCCAGCGTCTGAGCCGATCAGGATTCTGGACCAGCACGACTGCCCCGAGCCAGCCATTCAATCCAGCACTTCCAGTCCTCTTAAAGCCAGTATGTTTCCTCCCCAACTTTATTATTTCTTTTTATTTTCCCCCCCCAAAAAAACAAAAAAACCACAAAACCATGTCTGTTCTTGATGATGCTAACTTCCCCTCGATGGAGATGTTTTTCGAACGGTACGGTGATGACGGGGTCCAGGAACCCCGAGACGAACATTACATGCTGCATGAGGAATACGAGGAGACGTACGCAGCGCAATTTAGCTACGAATTCGGGCTGACTAAAGACACCAAGCTCCGTGATGTGTCCGAAACACCCATCAACTATGATGTCAGTGATTACTACTTTGACCCCCAGCTAGGGTTGGGCATCCACAAGGAGATCGGACTTGTGGTGGTGTTCCACCGAGACCCCCGGAAAAACTATATCATCCGGTCGCTTTTATTTGAGAGGTACGCACAGGCCATCATGTGCAAGATAACACTGTTCCGCGGGGCCTTGCAGTTGCTGCGGGCCAAGGACGAGCTCAGTCTGAAGCTTGTGCTCAGTTGCGAGATGGGTGAGCCAGGACTCGAACACGCCCGCGGCAAGCTCAGTCCGCAATGCATGCCGCTCCAGGCCGTGCCCATCCACTTTGGTTTTGGCTGCGGCTGCGGCATCATCAGAAGAAGGTATGGCGAGTTGAGTCGCCACTCATGCATCCAACGGGACTCGATCCGACCCATGGCGTTCCAAAAGTTCCCGCAGAGGTATGGTCTCCGGCCTGCCAACTTTGAGGTTTATTGTCCCACCGACGACGGCTATGTGGAGGCCCGCGCCAGGGATGGTCCTGCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCATTATCAGGGACTGCGACAACCGTCCAGAACCACCTGCGGACA

OrderedDict([('transl_table', ['12']),
             ('locus_tag', ['DEHA2G00110g']),
             ('old_locus_tag',
              ['DEHA0G00110g', 'DEHA-IPF1099', 'DEHA-CDS0121.1']),
             ('product', ['DEHA2G00110p']),
             ('note', ['no similarity']),
             ('db_xref',
              ['EnsemblGenomes-Gn:DEHA2G00110g',
               'EnsemblGenomes-Tr:CAG89967',
               'UniProtKB/TrEMBL:Q6BJT8']),
             ('protein_id', ['CAG89967.2']),
             ('translation',
              ['MVANGYHHKSSQRSSQRSSQRSSRSGFWTSTTAPSQPFNPALPVLLKPVCFLPNFIISFYFPPPKKQKNHKTMSVLDDANFPSMEMFFERYGDDGVQEPRDEHYMSHEEYEETYAAQFSYEFGSTKDTKLRDVSETPINYDVSDYYFDPQLGLGIHKEIGLVVVFHRDPRKNYIIRSLLFERYAQAIMCKITSFRGALQLSRAKDELSSKLVLSCEMGEPGLEHARGKLSPQCMPLQAVPIHFGFGCGCGIIRRRYGELSRHSCIQRDSIRPMAFQKFPQRYGLRPANFEVYCPTDDGYVEARARDGPASSSSSSSSSSLSGTATTVQNHSRTTTVPDSQHHPLVKKILQEVDREIVDIQVKGMKEANFVPKIAQLLVGAMTLFSANINHVNLAGLEVVRLFKGAANNTGRASNNFNRRPSTYFRAVQTYSTCVQYSKYMAELVYMVLMYGPKPAASLALTEMLDKYRDDEVVLEVNS

CR382133.2
CR382134.2
CR382135.2
CR382136.2
CR382137.2
CR382138.2
CR382139.2
